In [1]:
import bk.compute
import bk.load
import bk.plot

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Rectangle


import cv2
import yaml
import pickle

In [2]:
%matplotlib qt

In [3]:
path =  'Y://elevated_plus_maze/'
#config_path = 'Y:\elevated_plus_maze\elevated_plus_maze-BM-2020-12-01\config.yaml'

In [4]:
sessions = []
for subdirs, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('filtered.h5'): sessions.append(subdirs)
            
metadata = []
for subdirs, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.pickle'): metadata.append(os.path.join(subdirs,file))

del metadata[0:2]

videos = []
for subdirs, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.mp4') and not file.endswith('labeled.mp4'): videos.append(os.path.join(subdirs,file))

In [5]:
crops_x1 = []
crops_x2 = []
crops_y1 = []
crops_y2 = []
for m in metadata:
    with open(m,'rb') as f:
        data = pickle.load(f)
    x1,x2,y1,y2 = data['data']['cropping_parameters']
    crops_x1.append(x1)
    crops_x2.append(x2)
    crops_y1.append(y1)
    crops_y2.append(y2)


In [6]:
session = sessions[0]
os.chdir(session)
pos = bk.load.DLC_pos()


In [6]:
v  = 'Y:\elevated_plus_maze\Rat06-20201123\Rat06_201123_152703\Basler_acA1300-200uc__23039139__20201123_152706224.mp4'

In [7]:
cap = cv2.VideoCapture(v)
ret,frame = cap.read()
frame_cropped=frame[crops_y1[0]:crops_y2[0], crops_x1[0]:crops_x2[0]]
pause = True
fig,ax = plt.subplots()
plt.imshow(frame_cropped)

In [15]:
# colors = np.linspace(0,1,len(pos.as_units('s')))
viridis = matplotlib.cm.get_cmap('viridis')


In [12]:
plt.scatter(pos.as_units('s')['snout']['x'], pos.as_units('s')['snout']['y'], c = viridis(colors),s=10)
plt.show()

In [8]:
def onclick(event):
    global x,y,pause
    plt.cla()
    plt.imshow(frame)
    try:
        x
    except:
        x = [0,0]
    try:
        y
    except:
        y = [0,0]
    if (str(event.button) == "MouseButton.LEFT") or (event.button == 1):
        x[0], y[0] = event.xdata, event.ydata
    elif (str(event.button) == "MouseButton.RIGHT") or (event.button == 3):
        x[1], y[1] = event.xdata, event.ydata
    elif (str(event.button) == "MouseButton.MIDDLE") or (event.button == 2):
        fig.canvas.mpl_disconnect(cid)
        pause = False
    for i,j in zip(x,y):
        plt.axvline(i)
        plt.axhline(j)
        plt.draw()

In [9]:
positions_arms = {'open':np.empty([2,2],dtype='object'),'close':np.empty([2,2],dtype='object'),'center':np.empty([1,2],dtype='object')}
positions = ['up', 'down','left', 'right']

In [10]:
j = 0
for p in positions_arms:
    if p == 'center':
        positions_arms[p] = np.array([[(positions_arms['close'][0][0][1], positions_arms['close'][1][0][0]), (positions_arms['open'][0][1][1], positions_arms['open'][1][1][0])]],dtype='object')
    else:
        for i in range(positions_arms[p].shape[0]):
            try:
                del x,y
            except:
                pass

            cap = cv2.VideoCapture(v)
            ret,frame = cap.read()
            frame=frame[crops_y1[0]:crops_y2[0], crops_x1[0]:crops_x2[0]]
            pause = True
            fig,ax = plt.subplots()
            plt.imshow(frame)
            plt.title(p + positions[j])
            cid = fig.canvas.mpl_connect('button_press_event', onclick)

            while pause:
                plt.pause(0.1)

            plt.close('all')
            positions_arms[p][i] = [x,y]
            j += 1


In [14]:
np.empty([1,2],dtype='object')

array([[None, None]], dtype=object)

In [77]:
positions_arms['center'][0]

array([[400.5403225806451, 470.67741935483866],
       [282.9357258064516, 340.4841129032258]], dtype=object)

In [26]:
colors = ['r', 'b', 'green', 'orange', 'yellow']
j = 0
angle = np.linspace(0,360,20)
cap = cv2.VideoCapture(v)
ret,frame = cap.read()
frame_cropped=frame[crops_y1[0]:crops_y2[0], crops_x1[0]:crops_x2[0]]
pause = True
fig,ax = plt.subplots()


for i in range(15):
    for a in angle:
        plt.cla()
        plt.imshow(frame_cropped)
        ax = plt.gca()
        rectangles = []
        j = 0
        for p in positions_arms:
            for i in range(positions_arms[p].shape[0]):
                (x1,x2), (y1,y2) = positions_arms[p][i]
                rectangle = Rectangle((x1,y1), x2-x1, y2-y1,linewidth=1,edgecolor=viridis(a/360),facecolor=viridis(a/360),alpha = np.random.rand(), angle=np.random.rand()*360)
                ax.add_patch(rectangle)
                j += 1
        plt.draw()
        plt.pause(0.001)


In [46]:
cap = cv2.VideoCapture(v)
ret,frame = cap.read()
frame_cropped=frame[crops_y1[0]:crops_y2[0], crops_x1[0]:crops_x2[0]]
fig,ax = plt.subplots()
plt.imshow(frame_cropped)

In [66]:
(x1,x2),(y1,y2) = positions_arms['center']

s = Rectangle((x1,y1),x2-x1,y2-y1)

In [62]:
ax.add_patch(s)

In [63]:
s = Rectangle((0,0),250,250)

In [43]:
positions_arms['close'][1]


array([list([467.08064516129025, 880.7096774193548]),
       list([126.47604838709674, 426.80669354838705])], dtype=object)

In [22]:
positions_arms

{'open': array([[list([384.35483870967744, 474.2741935483871]),
         list([16.774435483871002, 284.7341129032258])],
        [list([398.741935483871, 488.6612903225806]),
         list([345.8792741935484, 615.6373387096774])]], dtype=object),
 'close': array([[list([-0.5, 400.5403225806451]),
         list([202.00830645161295, 376.45185483870966])],
        [list([467.08064516129025, 880.7096774193548]),
         list([126.47604838709674, 426.80669354838705])]], dtype=object),
 'center': array([[400.5403225806451, 467.08064516129025],
        [284.7341129032258, 345.8792741935484]], dtype=object)}

In [10]:
config_file = yaml.load(open(config_path,'r'))